In [ ]:
import xarray as xr
import pandas as pd
from functools import partial

OBS_DATA_PATH = "../data/raw/obs/"
OBS_COMBINED_DATA_PATH = "../data/processed/obs/"

In [ ]:
datetimes = pd.date_range("2022-01-01", "2024-09-02", freq="6h")
dates = xr.DataArray(
    [1] * len(datetimes), dims=["valid(UTC)"], coords={"valid(UTC)": datetimes}
)

In [ ]:
def _preprocess(x, dates):
    x = x.broadcast_like(dates)
    station = x.precip.attrs["station"]
    x = x.assign_coords({"station": station}).expand_dims("station")
    return x


partial_func = partial(_preprocess, dates=dates)

obs_combined = xr.open_mfdataset(
    f"{OBS_DATA_PATH}/*.nc",
    preprocess=partial_func,
    concat_dim="station",
    combine="nested",
)
obs_combined = obs_combined.precip

obs_combined.to_netcdf(f"{OBS_COMBINED_DATA_PATH}/obs.nc")